<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/LinearProgrammingPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming in Python

This notebook compares different Python alternatives to solve the following standar linear programming problems 

$$
\begin{align}
\underset{x}{\min} \quad & c^Tx \\
\text{s.t.} \quad & Ax \leq b\\
 & l \leq x \leq u\
\end{align}
$$

where $x\in\mathbb{R}^n$, $c\in\mathbb{R}^n$, $l\in\mathbb{R}^n$, $u\in\mathbb{R}^n$, $A\in\mathbb{R}^{m\times n}$, $b\in\mathbb{R}^m$. Elements of $A,b,c,l,u$ are randomly generating using normal or uniform distributions.

## Requirements

In [4]:
!pip install -q pyomo
!pip install -q gurobipy
!pip install -q docplex
!pip install -q pulp
!pip install -q cplex
!apt-get install -y -qq glpk-utils
import time
import pulp as pu
import numpy as np
import cplex
import pyomo.environ as pe
import pyomo.kernel as pk
import gurobipy as gp
from scipy.optimize import linprog
from pyomo.core.expr.numeric_expr import LinearExpression
from gurobipy import GRB
from docplex.mp.model import Model
glpk = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')

## Create random vectors and matrix of linear problem

In [5]:
# Number of variables (nvar) and constraints (ncon)
nvar = 1000
ncon = 200
# Random vectors and matrix of linear problem
c = np.random.normal(0,1,size=nvar)
A = np.random.normal(0,5,size=(ncon,nvar))
b = np.random.uniform(0,30,ncon)
u = np.random.uniform(10,20,nvar)
l = np.random.uniform(0,10,nvar)

## SCIPY ([link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html))

In [6]:
start = time.time()
# Solve problem
res = linprog(c, A_ub=A, b_ub=b, bounds=[(l[i],u[i]) for i in range(nvar)])
# Print results
print('obj =',res.fun)
print('time =',time.time()-start)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


obj = -1931.9489022809098
time = 9.434518337249756


## PULP ([link](https://coin-or.github.io/pulp/))

In [7]:
start = time.time()
# Model
m = pu.LpProblem('LinearProblem',pu.LpMinimize)
# Variables
x = [pu.LpVariable('x'+str(i+1), lowBound=l[i], upBound=u[i], cat="Continuous") for i in range(nvar)]
# Objective function
m += sum(c[i]*x[i] for i in range(nvar))
# Constraints
for j in range(ncon):
  m += sum(A[j,i]*x[i] for i in range(nvar)) <= b[j]
# Solve problem
m.solve()
# Princ results
print('obj =',pu.value(m.objective))
print('time =',time.time()-start)

obj = -1931.9534827988464
time = 57.86502552032471


## PYOMO + GLPK ([link](https://pyomo.readthedocs.io/en/stable/))



### Enviroment mode

In [8]:
start = time.time()
# Model
m = pe.ConcreteModel()
# Sets
m.i = pe.Set(initialize=range(nvar),ordered=True)
m.j = pe.Set(initialize=range(ncon),ordered=True)
# Variables
def bounds_x(m,i):
  return (l[i], u[i])
m.x = pe.Var(m.i,bounds=bounds_x)
# Objective function
def obj_rule(m):
  #return sum(c[i]*m.x[i] for i in m.i)
  #return pe.quicksum(c[i]*m.x[i] for i in m.i)
  return LinearExpression(constant=0, linear_coefs=[c[i] for i in m.i], linear_vars=[m.x[i] for i in m.i])
m.obj = pe.Objective(rule=obj_rule)
# Constraints
def con_rule(m,j):
  #return sum(A[j,i]*m.x[i] for i in m.i) <= b[j]
  #return pe.quicksum(A[j,i]*m.x[i] for i in m.i) <= b[j]
  return LinearExpression(constant=0, linear_coefs=[A[j,i] for i in m.i], linear_vars=[m.x[i] for i in m.i]) <= b[j]
m.con = pe.Constraint(m.j,rule=con_rule)
# Solve the problem
glpk.solve(m)
# Print results
print('obj =',m.obj())
print('time =',time.time()-start)

obj = -1931.9534884056823
time = 2.2824199199676514


### Kernel mode

In [9]:
start = time.time()
# Model
m = pk.block()
# Sets
m.i = [i for i in range(nvar)]
m.j = [j for j in range(ncon)]
# Variables
m.x = pk.variable_list()
for i in m.i:
  m.x.append(pk.variable(lb=l[i],ub=u[i]))
# Objective function
m.obj = pk.objective(LinearExpression(constant=0, linear_coefs=[c[i] for i in m.i], linear_vars=[m.x[i] for i in m.i]))
# Constraints
m.con = pk.matrix_constraint(A, ub=b, x=[m.x[i] for i in m.i])
# Solve the problem
glpk.solve(m)
# Print results
print('obj =',m.obj())
print('time =',time.time()-start)

obj = -1931.9534884056823
time = 2.5167713165283203


## GUROBIPY ([link](https://pypi.org/project/gurobipy/))

### Matrix notation

In [10]:
 start = time.time()
 # Model
 m = gp.Model()
 # Variables
 x = m.addMVar(shape=nvar,lb=l,ub=u,vtype=GRB.CONTINUOUS)
 # Objective function
 m.setObjective(c @ x, GRB.MINIMIZE)
 # Constraints
 m.addConstr(A @ x <= b)
 # Solve the problem
 m.optimize()
 # Print results
 print('obj =',m.ObjVal)
 print('time =',time.time()-start)


Restricted license - for non-production use only - expires 2022-01-13
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 200 rows, 1000 columns and 200000 nonzeros
Model fingerprint: 0x180fd1dc
Coefficient statistics:
  Matrix range     [8e-05, 2e+01]
  Objective range  [4e-04, 4e+00]
  Bounds range     [3e-03, 2e+01]
  RHS range        [5e-02, 3e+01]
Presolve time: 0.09s
Presolved: 200 rows, 1000 columns, 200000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.4296535e+06   7.927222e+06   0.000000e+00      0s
     549   -1.9319535e+03   0.000000e+00   0.000000e+00      0s

Solved in 549 iterations and 0.26 seconds
Optimal objective -1.931953488e+03
obj = -1931.9534884057082
time = 0.3106820583343506


### Constraint notation

In [11]:
start = time.time()
# Model
m = gp.Model()
# Variables
x = m.addVars(nvar,lb=l,ub=u,vtype=GRB.CONTINUOUS)
# Objective function
m.setObjective(gp.LinExpr(c,x.values()),GRB.MINIMIZE)
# Constraints
m.addConstrs((gp.LinExpr(A[j,:],x.values()) <= b[j] for j in range(ncon)))
# Solve the problem
m.optimize()
# Print results
print('obj = ',m.ObjVal)
print('time =',time.time()-start)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 200 rows, 1000 columns and 200000 nonzeros
Model fingerprint: 0x180fd1dc
Coefficient statistics:
  Matrix range     [8e-05, 2e+01]
  Objective range  [4e-04, 4e+00]
  Bounds range     [3e-03, 2e+01]
  RHS range        [5e-02, 3e+01]
Presolve time: 0.10s
Presolved: 200 rows, 1000 columns, 200000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.4296535e+06   7.927222e+06   0.000000e+00      0s
     549   -1.9319535e+03   0.000000e+00   0.000000e+00      0s

Solved in 549 iterations and 0.26 seconds
Optimal objective -1.931953488e+03
obj =  -1931.9534884057082
time = 0.48479795455932617


## DOCPLEX ([link](https://pypi.org/project/docplex/))

In [12]:
start = time.time()
# Model
m = Model(name='LP')
# Sets
m.i = range(nvar)
m.j = range(ncon)
# Variables
x = m.continuous_var_list(m.i,lb=l,ub=u)
# Objective function
m.minimize(m.sum(c[i]*x[i] for i in m.i))
# Constraints
for j in m.j:
  m.add_constraint(m.sum(A[j,i]*x[i] for i in m.i) <= b[j])
# Solve the problem
res = m.solve()
# Print results
print('obj =',res.objective_value)
print('time =',time.time()-start)

obj = -1931.9534884057075
time = 1.2735075950622559
